In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.linalg as linalg
from numba import jit

In [2]:
from scipy.stats import norm
def Black_Scholes_Call(S, K, r, vol, tau):
    """ 
    Black Scholes Model for European Call
    """
    d1 = (np.log(S / K) + (r + ((vol**2)/2.)*tau)) / (vol*np.sqrt(tau))
    d2 = d1 - vol*np.sqrt(tau)
    V = S * norm.cdf(d1) - np.exp(-r*tau) * K * norm.cdf(d2)
    
    return V

For this equation, we conclude that
\begin{align}
    \vec{a_{-1}} &= \frac{1}{2}\sigma^{2}\frac{\Delta\tau}{\Delta X^{2}} - \frac{\Delta\tau}{2\Delta X}(r-\frac{1}{2}\sigma^{2}) \\
    \vec{a_{0}} &=  1- \sigma^{2}\frac{\Delta\tau}{\Delta X^{2}} - r\Delta\tau\\
    \vec{a_{+1}} &=  \frac{\Delta\tau}{2\Delta X}(r-\frac{1}{2}\sigma^{2})+\frac{1}{2}\sigma^{2}\frac{\Delta\tau}{\Delta X^{2}}
\end{align}

In [ ]:
smax = 10000
smin = 1
xaxis =  np.linspace(np.log(smin), np.log(smax), M)
dx = (xaxis[-1] - xaxis[0]) / M

In [31]:
dt = T/N
x_max = np.log(S) + 12*sigma*np.sqrt(T);
x_min = np.log(S) - 12*sigma*np.sqrt(T);
dx = (x_max - x_min)/M;
ss = sigma**2
dxx = dx**2

up = ((0.5*(sigma**2) * dt / dx**2)    +    ((r-0.5*(sigma**2)) *dt / (2*dx)))

ad = (dt/2)* ((r - 1/2*ss)/dx - ss/dxx)

In [504]:
def FD(S, K, r, sigma, T, N, M, scheme="FCTS"):
    # Compute delta T
    dt = T/N
        
    s_max = 10000
    s_min = 1
    X = np.linspace(np.log(s_min), np.log(s_max), M)
    dx = (X[-1] - X[0])/(M-1);

    # Generate stock prices on log scale
    # Generate T * S grid
    V = np.zeros((N,M))

    # Put the discounted values into the first column
    V[0, :] = [max(np.exp(s)-K,0) for s in X]
    V[0, -1] = s_min
    V[0, 0] = s_max
    # Fucking constants
    ss = sigma * sigma
    dxx = dx*dx
    
    # a-1, a0, a+1
    if scheme=="FCTS":
        ad = ((0.5*(sigma**2) * dt / dx**2)    -    ((r-0.5*(sigma**2)) *dt / (2*dx)))
        a0 = (1-((sigma**2) * dt / dx**2) - r * dt)
        au = ((0.5*(sigma**2) * dt / dx**2)    +    ((r-0.5*(sigma**2)) *dt / (2*dx)))
    else:
        ad = (ss*dt)/(4*dxx) - ((r-0.5*ss) * (dt/4*dx))
        a0 = (-2*((ss*dt)/(4*dxx)) + (r*dt/2) + 1)
        au = (ss*dt)/(4*dxx) + ((r-0.5*ss) * (dt/4*dx))
        
        bd
        b0
        bu
        
    # Generate matrix A
    A = np.zeros((M,M))
    
    np.fill_diagonal(A[1:], ad)
    np.fill_diagonal(A[:,1:], au)
    np.fill_diagonal(A, a0)
    
    A[0,0], A[0,1], A[-1,-2], A[-1,-1] = 1, 0, 1,1
    
    # V[n+1] = A * V[m]
    #return A, V
    for i in range(1,N):
        V[i] = np.matmul(A, V[i-1])

    return A, V , X

In [501]:
sigma = 0.3
K = 110
S = 130
r = 0.04
T = 1
N = 1500# time points
M =  300# space points

In [502]:
Black_Scholes_Call(S, K, r, sigma, T)

29.38930249574946

In [493]:
result = FD(S, K, r, sigma, T, N, M, scheme="FCTS")

In [503]:
result[2][abs(np.exp(result[2])-S)==abs(np.exp(result[2])-S).min()]

nearest_idx = np.where(abs(np.exp(result[2])-S)==abs(np.exp(result[2])-S).min())[0]
result[1][-1][nearest_idx]

array([29.33014133])